In [ ]:
import numpy as np

In [ ]:
def variance_covariance(beta, model, T, returns, quantile, sigmas):
    """
    Use Manganelli's matlab code as a reference
    
    :param: beta (np.array)
    :param: model (str)
    :param: T (int): number of in-sample obs.
    :param: returns (np.array): returns
    :param: quantile (float)
    :param: sigma (np.array): VaR
    """
    # Compute the quantile residuals
    residuals = returns - sigmas
    
    # Set up the bandwidth for the KNN algorithm
    sorted_result = np.sort(residuals)
    k = 40 if quantile == 0.01 else 60
    bandwidth = sorted_result[40]
    t = 0
    
    # initialize vectors
    derivative1 = np.zeros((T, 1))
    derivative2 = np.zeros((T, 1))
    derivative3 = np.zeros((T, 1))
    derivative4 = np.zeros((T, 1))
    
    D = np.zeros(beta.shape[0])
    A = D
    
    if model == 'adaptive':
        gradient = np.zeros((T, 1))
        pass
    elif model == 'asymmetric':
        gradient = np.zeros((T, 4))
        pass
    elif model == 'symmetric':
        gradient = np.zeros((T, 3))
        
        for i in range(1, T):
            derivative1[i] = 1 + beta[1] * derivative1[i-1]
            derivative2[i] = sigmas[i-1] + beta[1] * derivative2[i-1]
            derivative3[i] = beta[1] * derivative3[i-1] + abs(returns[i-1])
            
            gradient[i, 0] = derivative1[i]
            gradient[i, 1] = derivative2[i]
            gradient[i, 2] = derivative3[i]
            
            A = A + gradient[i, :].T @ gradient[i, :] # this one should be a matrix
            
            if abs(residuals[i]) <= bandwidth:
                t += 1
                D = D + gradient[i, :].T @ gradient[i, :] # this one should be a matrix
                
    elif model == 'igarch':
        gradient = np.zeros((T, 3))
        pass
    else:
        raise ValueError('Wrong model!')
        
    t_std_err = t # check the bandwidth
    A = A / T
    D = D / (2 * bandwidth * T)
    
    inv_D = np.linalg.inv(D)
    vc_matrix = (quantile * (1 - quantile) / T) * (inv_D @ A @ inv_D)
    
    return vc_matrix, D, gradient

In [ ]:
vc_matrix, D, gradient = variance_covariance(beta, model, T, y ,quantile, sigma)

In [ ]:
standard_error = np.sqrt(np.diag(vc_matrix))
# coefficient_pval = normcdf(-abs(beta))

Hit'X * inv(MM') * X'Hit / THETA*(1-THETA), where M = X' - inv(T)*(X'*H * grad(f)) * inv(D) * grad(f)'

In [ ]:
X = np.zeros((T, p))
Hit = np.zeros((T, p))

In [ ]:
gradient = np.array([])

dev = y - y_hat
idx = np.where(np.abs(dev) < bandwidth)

D = np.sum(gradient[idx] * gradient[idx])
D *= 1 / (2 * T * bandwidth)

M = X.T - (np.sum(X[idx] * gradient[idx]) / (2 * T * bandwidth)) @ np.linalg.inv(D) @ gradient.T

In [ ]:
(Hit.T @ X * np.linalg.inv(M @ M.T) * X.T * Hit) / (theta * (1 - theta)) # follows Chi-square